#  Product Analysis

This analysis focuses on identifying the most popular products based on quantity sold. Understanding product popularity helps optimize inventory management, marketing efforts, and sales strategies.

---

## What We Will Explore:

### 1. Product Popularity Metrics
- **Total Quantity Sold**: Sum of all units sold per product.
- **Top-Selling Products**: Products with the highest sales volume.

---

### 2. Import Cleaned Data from ETL Pipeline

We use the cleaned dataset (`df_cleaned`) that was prepared in the ETL pipeline. The following steps were already applied:
- Removed rows with missing `CustomerID` or `Description`.
- Removed cancelled orders (Invoices starting with 'C').
- Excluded records with negative or zero `Quantity` or `UnitPrice`.
- Converted `InvoiceDate` to datetime format.
- Created new columns: `TotalPrice`, `Year`, `Month`, `Day`, `Hour`, `Weekday`.
- Renamed `Description` to `ProductName`.
- Cleaned `ProductName` by stripping whitespace and converting to lowercase.
- Removed duplicate records.

These preprocessing steps ensure we analyze clean, accurate, and consistent data.

---


## 3. Visualizations

To effectively understand product trends and identify bestsellers, we will use:

- **Seaborn**: A horizontal bar chart that displays the top 10 products ranked by the total quantity sold, using color to enhance readability and clarity.



## Importing Libraries

Before we begin our product analysis, we need to import the essential Python libraries that will help us load, process, and visualize the data.

### Libraries Used:

- **pandas**: For data manipulation and analysis (e.g., grouping products, aggregating sales).
- **numpy**: For numerical operations and handling arrays.
- **matplotlib.pyplot**: A core Python library for creating static visualizations like bar charts.
- **seaborn**: A higher-level visualization library built on top of matplotlib, useful for creating aesthetically pleasing charts such as horizontal bar plots.

### Display Settings:

- `%matplotlib inline`: Ensures that Matplotlib charts appear directly within the notebook cells.
- `plt.style.use('ggplot')`: Applies a clean and visually appealing style to Matplotlib plots.
- `sns.set(style='whitegrid')`: Sets the background style of Seaborn plots to show light gridlines for better readability.

These libraries will provide the necessary tools for data aggregation, cleaning, and visual storytelling throughout this product analysis.


In [21]:
# Importing Libraries for Product Analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings for plots
%matplotlib inline
plt.style.use('ggplot')
sns.set(style='whitegrid')


In [22]:
# Step 1: Load cleaned data 
df = pd.read_csv('../data/cleaned/cleaned_online_retail.csv')
df.head



/var/folders/2b/6mjnm51s11l2r_38ms04m1jc0000gn/T/ipykernel_22978/1228718833.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/cleaned/cleaned_online_retail.csv')


<bound method NDFrame.head of        InvoiceNo StockCode                          ProductName  Quantity  \
0         536365    85123A   white hanging heart t-light holder         6   
1         536365     71053                  white metal lantern         6   
2         536365    84406B       cream cupid hearts coat hanger         8   
3         536365    84029G  knitted union flag hot water bottle         6   
4         536365    84029E       red woolly hottie white heart.         6   
...          ...       ...                                  ...       ...   
524873    581587     22613          pack of 20 spaceboy napkins        12   
524874    581587     22899          children's apron dolly girl         6   
524875    581587     23254         childrens cutlery dolly girl         4   
524876    581587     23255      childrens cutlery circus parade         4   
524877    581587     22138         baking set 9 piece retrospot         3   

                InvoiceDate  UnitPrice  Custo

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 524878 entries, 0 to 524877
Data columns (total 14 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    524878 non-null  object 
 1   StockCode    524878 non-null  object 
 2   ProductName  524878 non-null  object 
 3   Quantity     524878 non-null  int64  
 4   InvoiceDate  524878 non-null  object 
 5   UnitPrice    524878 non-null  float64
 6   CustomerID   524878 non-null  int64  
 7   Country      524878 non-null  object 
 8   TotalPrice   524878 non-null  float64
 9   Year         524878 non-null  int64  
 10  Month        524878 non-null  int64  
 11  Day          524878 non-null  int64  
 12  Hour         524878 non-null  int64  
 13  Weekday      524878 non-null  object 
dtypes: float64(2), int64(6), object(6)
memory usage: 56.1+ MB


In [25]:
# Convert 'InvoiceDate' from string/object to datetime format
# This enables date operations like sorting and filtering
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Convert 'Invoice' from object to numeric type
# errors='coerce' turns invalid values into NaN instead of error
# .astype('Int64') ensures integer type that supports missing values (NaN)
df['InvoiceNo'] = pd.to_numeric(df['InvoiceNo'], errors='coerce').astype('Int64')


In [27]:
#to validate
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 524878 entries, 0 to 524877
Data columns (total 14 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    524877 non-null  Int64         
 1   StockCode    524878 non-null  object        
 2   ProductName  524878 non-null  object        
 3   Quantity     524878 non-null  int64         
 4   InvoiceDate  524878 non-null  datetime64[ns]
 5   UnitPrice    524878 non-null  float64       
 6   CustomerID   524878 non-null  int64         
 7   Country      524878 non-null  object        
 8   TotalPrice   524878 non-null  float64       
 9   Year         524878 non-null  int64         
 10  Month        524878 non-null  int64         
 11  Day          524878 non-null  int64         
 12  Hour         524878 non-null  int64         
 13  Weekday      524878 non-null  object        
dtypes: Int64(1), datetime64[ns](1), float64(2), int64(6), object(4)
memory usage: 56.6+ 